# Projet


## Collecte de données
Nous commencons par créer notre dossier images, dans le repertoire courant.
Nous téléchargons ensuite nos cents images depuis la banque d'images : 

In [6]:
!pip3 install --upgrade kaggle

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/bin/python3 -m pip install --upgrade pip' command.


In [1]:
import os
os.environ['KAGGLE_USERNAME'] = 'thouveninnicolas'
os.environ['KAGGLE_KEY'] = 'e82a9a45b62d43d75bae2278615230af'

In [2]:
import kaggle

kaggle.api.authenticate()

kaggle.api.dataset_download_files('Dogs vs. Cats', path='./', unzip=True)

In [ ]:
import kaggle

kaggle.api.authenticate()

kaggle.api.competitions_list()